# Collaboration and Competition

---

In this notebook, you will learn how to use the Unity ML-Agents environment for the third project of the [Deep Reinforcement Learning Nanodegree](https://www.udacity.com/course/deep-reinforcement-learning-nanodegree--nd893) program.

### 1. Start the Environment

We begin by importing the necessary packages.  If the code cell below returns an error, please revisit the project instructions to double-check that you have installed [Unity ML-Agents](https://github.com/Unity-Technologies/ml-agents/blob/master/docs/Installation.md) and [NumPy](http://www.numpy.org/).

In [ ]:
from unityagents import UnityEnvironment
import numpy as np
from collections import deque
import importlib
import torch
import matplotlib.pyplot as plt
%matplotlib inline

# Self-created modules
import ddpg_agent

Next, we will start the environment!  **_Before running the code cell below_**, change the `file_name` parameter to match the location of the Unity environment that you downloaded.

- **Mac**: `"path/to/Tennis.app"`
- **Windows** (x86): `"path/to/Tennis_Windows_x86/Tennis.exe"`
- **Windows** (x86_64): `"path/to/Tennis_Windows_x86_64/Tennis.exe"`
- **Linux** (x86): `"path/to/Tennis_Linux/Tennis.x86"`
- **Linux** (x86_64): `"path/to/Tennis_Linux/Tennis.x86_64"`
- **Linux** (x86, headless): `"path/to/Tennis_Linux_NoVis/Tennis.x86"`
- **Linux** (x86_64, headless): `"path/to/Tennis_Linux_NoVis/Tennis.x86_64"`

For instance, if you are using a Mac, then you downloaded `Tennis.app`.  If this file is in the same folder as the notebook, then the line below should appear as follows:
```
env = UnityEnvironment(file_name="Tennis.app")
```

In [ ]:
env = UnityEnvironment(file_name="./Tennis_Windows_x86_64/Tennis.exe")

Environments contain **_brains_** which are responsible for deciding the actions of their associated agents. Here we check for the first brain available, and set it as the default brain we will be controlling from Python.

In [ ]:
# get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]

### 2. Examine the State and Action Spaces

In this environment, two agents control rackets to bounce a ball over a net. If an agent hits the ball over the net, it receives a reward of +0.1.  If an agent lets a ball hit the ground or hits the ball out of bounds, it receives a reward of -0.01.  Thus, the goal of each agent is to keep the ball in play.

The observation space consists of 8 variables corresponding to the position and velocity of the ball and racket. Two continuous actions are available, corresponding to movement toward (or away from) the net, and jumping. 

Run the code cell below to print some information about the environment.

In [ ]:
# reset the environment
env_info = env.reset(train_mode=True)[brain_name]

# number of agents 
num_agents = len(env_info.agents)
print('Number of agents:', num_agents)

# size of each action
action_size = brain.vector_action_space_size
print('Size of each action:', action_size)

# examine the state space 
states = env_info.vector_observations
state_size = states.shape[1]
print('There are {} agents. Each observes a state with length: {}'.format(states.shape[0], state_size))
print('The state for the first agent looks like:', states[0])

### 3. Take Random Actions in the Environment

In the next code cell, you will learn how to use the Python API to control the agents and receive feedback from the environment.

Once this cell is executed, you will watch the agents' performance, if they select actions at random with each time step.  A window should pop up that allows you to observe the agents.

Of course, as part of the project, you'll have to change the code so that the agents are able to use their experiences to gradually choose better actions when interacting with the environment!

In [ ]:
for i in range(1, 6):                                      # play game for 5 episodes
    env_info = env.reset(train_mode=False)[brain_name]     # reset the environment    
    states = env_info.vector_observations                  # get the current state (for each agent)
    scores = np.zeros(num_agents)                          # initialize the score (for each agent)
    while True:
        actions = np.random.randn(num_agents, action_size) # select an action (for each agent)
        actions = np.clip(actions, -1, 1)                  # all actions between -1 and 1
        env_info = env.step(actions)[brain_name]           # send all actions to tne environment
        next_states = env_info.vector_observations         # get next state (for each agent)
        rewards = env_info.rewards                         # get reward (for each agent)
        dones = env_info.local_done                        # see if episode finished
        scores += env_info.rewards                         # update the score (for each agent)
        states = next_states                               # roll over states to next time step
        if np.any(dones):                                  # exit loop if episode finished
            break
    print('Score (max over agents) from episode {}: {}'.format(i, np.max(scores)))

When finished, you can close the environment.

In [ ]:
env.close()

### 4. It's Your Turn!

Now it's your turn to train your own agent to solve the environment!  When training the environment, set `train_mode=True`, so that the line for resetting the environment looks like the following:
```python
env_info = env.reset(train_mode=True)[brain_name]
```

In [ ]:
# Target parameters as provided in "The Environment - Introduction"-lesson
average_target_score = 0.5
# For testing of code functionality
#average_target_score = 0.0
average_episode_memory = 100

update_time_unit = 1
n_updates_per_update_time_unit = 4

start_learing_episode = 200
# For testing of code functionality
#start_learing_episode = 1

In [ ]:
importlib.reload(ddpg_agent)

agent = ddpg_agent.Agents(state_size=state_size, action_size=action_size, random_seed=2, n_agents=num_agents)

def ddpg(n_episodes=2000, max_t=1000, print_every=average_episode_memory, n_actions=action_size, n_agents=num_agents):
    scores_deque = deque(maxlen=average_episode_memory)
    scores = []
    for i_episode in range(1, n_episodes+1):
        # Reset environment and get states for all agents 
        env_info = env.reset(train_mode=True)[brain_name]
        states = env_info.vector_observations
        assert states.shape[0]==n_agents, f"In episode {i_episode} the number of states does not match the number of agents"
        agent.reset()
        agent_scores = np.zeros(n_agents)     
        for t in range(max_t):
            actions = np.zeros((n_agents, n_actions))
            for i_agent in range(n_agents):
                actions[i_agent] = agent.act(states[i_agent])

            env_info = env.step(actions)[brain_name] 
            next_states = env_info.vector_observations
            rewards = env_info.rewards
            dones = env_info.local_done
            
            # Add experiences from all agents to memory
            for i_agent in range(n_agents):
                agent.add_experience_to_memory(states[i_agent], 
                                         actions[i_agent], 
                                         rewards[i_agent], 
                                         next_states[i_agent],
                                         dones[i_agent]
                                        )   
            
            # Only start updates after a certain amout of episodes were played in order to have the replay buffer filled
            if i_episode >= start_learing_episode:
                # Perform updates at every 1 time unit
                if t % update_time_unit == 0:
                    for i in range(n_updates_per_update_time_unit):
                        for j in range(n_agents):
                            agent.learn_from_replay(j)
                
            agent_scores += rewards    
            states = next_states
            
            if np.any(dones):
                break    
             
        # According to environment instructions 
        # "After each episode, we add up the rewards that each agent received (without discounting), 
        #  to get a score for each agent. 
        #  This yields 2 (potentially different) scores. We then take the maximum of these 2 scores."
        score = np.max(agent_scores)
        scores_deque.append(score)
        scores.append(score)     
        print(f'\rEpisode {i_episode}\tAverage Score: {np.mean(scores_deque):.5f}\tCurrent Score {score:.5f}', end="")
        if i_episode % print_every == 0:
            print(f'\rEpisode {i_episode}\tAverage Score: {np.mean(scores_deque):.5f}')  
        if np.mean(scores_deque)>=average_target_score:
            print(f'\nEnvironment solved in {i_episode:d} episodes!\tAverage Score: {np.mean(scores_deque):.5f}')
            torch.save(agent.actor_local.state_dict(), 'checkpoint_actor.pth')
            torch.save(agent.critic_local[0].state_dict(), 'checkpoint_critic_0.pth')
            torch.save(agent.critic_local[1].state_dict(), 'checkpoint_critic_1.pth')
            break  
    return scores
        
        
scores = ddpg(n_episodes=3000, max_t=100000)

# plot the scores
fig = plt.figure()
ax = fig.add_subplot(111)
plt.plot(np.arange(len(scores)), scores)
plt.ylabel('Score')
plt.xlabel('Episode #')
plt.savefig('./plots/plot_of_rewards.png', dpi=200)
plt.show()

In [ ]:
env.close()